# Generate Low Test Models

This notebook is a streamlined notebook for generating minima of low test accuracy through three different means:
- Dataset Poisoning
- Adding Noise to Data
- Decreasing Dataset Sizes

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import time

# Importing our existing funcs
import os
import sys
from pathlib import Path
parent_dir = Path.cwd().parent.parent.parent
sys.path.append(str(parent_dir))
# Import modules

from train_funcs import ( train, evaluate)

from dataset_funcs import ( prepare_datasets,
                            save_model, save_dataset,
                            load_model, load_dataset )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Input Parameters

In [2]:
# Base Input Parameters
# Training Configuration
data_seed = 4
model_seed = 4
epochs = 500

# Training Data Configuration
base_data_size = 60
dataset_type = "data"
dataset_quantities = [0, 600 - 60, 6000 - 60, 60000 - 60]

# Base output directory
base_output_dir = ""

# Whether or not to save the generated models and datasets
save_generated_dataset = True
save_generated_models = True


## MNIST Data Generation

We grab MNIST, and partition it into train and test datasets.

In [3]:
import torch.nn.functional as F
from torchvision import datasets, transforms

# Define transform (convert to tensor + normalize)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load dataset (not a tensor yet, just a Dataset object)
train_dataset = datasets.MNIST('../../../data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST('../../../data', train=False, transform=transform)

# Convert entire dataset into big tensors
train_images = torch.stack([img for img, _ in train_dataset])   # shape: [60000, 1, 28, 28]
train_labels = torch.tensor([label for _, label in train_dataset])  # shape: [60000]

test_images  = torch.stack([img for img, _ in test_dataset])    # shape: [10000, 1, 28, 28]
test_labels  = torch.tensor([label for _, label in test_dataset])   # shape: [10000]

print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

x_base = train_images.to(device)
y_base = train_labels.to(device)
x_test = test_images.to(device)
y_test = test_labels.to(device)

batch_size = len(x_base)
criterion = nn.CrossEntropyLoss()

torch.Size([60000, 1, 28, 28]) torch.Size([60000])
torch.Size([10000, 1, 28, 28]) torch.Size([10000])


## Model

The model we train on this data. This will vary from problem to problem.

In [4]:
# Model Definition
class NetMLP(nn.Module):
    def __init__(self, seed=None):
        super(NetMLP, self).__init__()
        if seed is not None:
            self.seed = seed
            with torch.random.fork_rng():
                torch.manual_seed(seed)
                self._initialize_layers()
        else:
            self._initialize_layers()

    def _initialize_layers(self):
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)        # Flatten input (B, 1, 28, 28) → (B, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # No softmax here
        return x

## Training

We generate the various datasets used to train our models here, before training them. We record the losses, and what each model was trained on.

In [5]:
x_base_train, y_base_train, x_additional, y_additional = prepare_datasets(x_base=x_base, y_base=y_base,
                                                                          dataset_type=dataset_type, 
                                                                          dataset_quantities=dataset_quantities,
                                                                          base_data_size=base_data_size,
                                                                          data_seed=data_seed,
                                                                          seed_1 = None,
                                                                          seed_2 = None)

x_base_train = x_base_train.to(device)
y_base_train = y_base_train.to(device)
x_additional = x_additional.to(device)
y_additional = y_additional.to(device)

x_test = x_test.to(device)
y_test = y_test.to(device)

criterion = nn.CrossEntropyLoss()

all_models = []

for additional_data in dataset_quantities:
    x_train = torch.cat([x_base_train, x_additional[:additional_data]], dim=0)
    y_train = torch.cat([y_base_train, y_additional[:additional_data]], dim=0)

    # Train model - this will vary depending on the problem
    torch.manual_seed(model_seed)
    model = NetMLP().to(device)
    optimizer = optim.AdamW(model.parameters(), lr=1e-3)
    model = model.to(device)

    batch_size = len(x_train)
    train_losses, train_accs, test_losses, test_accs = train(
        model, x_train, y_train, x_test, y_test, 
        criterion, optimizer, epochs=epochs, batch_size = batch_size, verbose_every=10
    )
    
    # Store trained model and metrics
    trained_model = {
        'model': model,
        'train_losses': train_losses,
        'train_accs': train_accs,
        'test_losses': test_losses,
        'test_accs': test_accs,
        'additional_data': additional_data,
        'dataset_type': dataset_type,
    }
    all_models.append(trained_model)
    
    print(f"Completed training with {additional_data} additional samples of {dataset_type}")

    # Free the assembled dataset (optional but good for large GPU datasets)
    del x_train, y_train
    torch.cuda.empty_cache()

Epoch 1/500: Train Loss 2.3077, Acc 0.1000 | Test Loss 2.2269, Acc 0.1762


Epoch 10/500: Train Loss 0.4648, Acc 0.9500 | Test Loss 1.1490, Acc 0.6390


Epoch 20/500: Train Loss 0.0124, Acc 1.0000 | Test Loss 1.0407, Acc 0.6784


Epoch 30/500: Train Loss 0.0009, Acc 1.0000 | Test Loss 1.2381, Acc 0.6859


Epoch 40/500: Train Loss 0.0003, Acc 1.0000 | Test Loss 1.3606, Acc 0.6855


...

Epoch 380/500: Train Loss 0.0048, Acc 0.9998 | Test Loss 0.0863, Acc 0.9781
Epoch 390/500: Train Loss 0.0044, Acc 0.9998 | Test Loss 0.0871, Acc 0.9779
Epoch 400/500: Train Loss 0.0040, Acc 0.9999 | Test Loss 0.0880, Acc 0.9779


Epoch 410/500: Train Loss 0.0037, Acc 0.9999 | Test Loss 0.0888, Acc 0.9781
Epoch 420/500: Train Loss 0.0034, Acc 0.9999 | Test Loss 0.0897, Acc 0.9780
Epoch 430/500: Train Loss 0.0032, Acc 1.0000 | Test Loss 0.0905, Acc 0.9778


Epoch 440/500: Train Loss 0.0029, Acc 1.0000 | Test Loss 0.0913, Acc 0.9781
Epoch 450/500: Train Loss 0.0027, Acc 1.0000 | Test Loss 0.0921, Acc 0.9782
Epoch 460/500: Train Loss 0.0025, Acc 1.0000 | Test Loss 0.0929, Acc 0.9783


Epoch 470/500: Train Loss 0.0024, Acc 1.0000 | Test Loss 0.0937, Acc 0.9783
Epoch 480/500: Train Loss 0.0022, Acc 1.0000 | Test Loss 0.0944, Acc 0.9783
Epoch 490/500: Train Loss 0.0021, Acc 1.0000 | Test Loss 0.0951, Acc 0.9783


Epoch 500/500: Train Loss 0.0019, Acc 1.0000 | Test Loss 0.0959, Acc 0.9784
Completed training with 59940 additional samples of data


## Training Summary

In [6]:
# Generic summary of training results
print ("True Generalization:")
for model_data in all_models:
    model = model_data['model']
    model_additional_data = model_data['additional_data']
    
    test_loss, test_accuracy = evaluate(model, x_test, y_test, criterion)

    print(f"{model_additional_data} \t Model : "
            f"Test Loss {test_loss:.4f}, Acc {test_accuracy:.4f} ")

for additional_data in dataset_quantities:
    x_train = torch.cat([x_base_train, x_additional[:additional_data]], dim=0)
    y_train = torch.cat([y_base_train, y_additional[:additional_data]], dim=0)

    print(f"\nModel diagnostics for {dataset_type}: ", additional_data)
    
    for model_data in all_models:
        model = model_data['model']
        model_additional_data = model_data['additional_data']
        
        train_loss, train_accuracy = evaluate(model, x_train, y_train, criterion)

        print(f"{model_additional_data} \t Poison Model : "#Clean Train Loss {clean_loss:.4f}, Acc {clean_accuracy:.4f} | "
                f" Train Loss {train_loss:.4f}, Acc {train_accuracy:.4f} ")

True Generalization:
0 	 Model : Test Loss 1.4419, Acc 0.6943 
540 	 Model : Test Loss 0.8086, Acc 0.8642 
5940 	 Model : Test Loss 0.3219, Acc 0.9440 
59940 	 Model : Test Loss 0.0959, Acc 0.9784 

Model diagnostics for data:  0
0 	 Poison Model :  Train Loss 0.0000, Acc 1.0000 
540 	 Poison Model :  Train Loss 0.0001, Acc 1.0000 
5940 	 Poison Model :  Train Loss 0.0003, Acc 1.0000 
59940 	 Poison Model :  Train Loss 0.0012, Acc 1.0000 

Model diagnostics for data:  540
0 	 Poison Model :  Train Loss 1.4178, Acc 0.6917 
540 	 Poison Model :  Train Loss 0.0001, Acc 1.0000 
5940 	 Poison Model :  Train Loss 0.0003, Acc 1.0000 
59940 	 Poison Model :  Train Loss 0.0025, Acc 1.0000 

Model diagnostics for data:  5940
0 	 Poison Model :  Train Loss 1.4940, Acc 0.6825 
540 	 Poison Model :  Train Loss 0.7278, Acc 0.8773 
5940 	 Poison Model :  Train Loss 0.0003, Acc 1.0000 
59940 	 Poison Model :  Train Loss 0.0020, Acc 1.0000 

Model diagnostics for data:  59940
0 	 Poison Model :  Train 

## Model Saving

In [7]:
# === Save dataset ===
if save_generated_dataset: #not always needed, eg, MNIST and CIFAR
    save_dataset(
        folder="models_and_data",
        filename="dataset.pt",
        x_base_train=x_base_train,
        y_base_train=y_base_train,
        x_additional=x_additional,
        y_additional=y_additional,
        x_test=x_test,
        y_test=y_test,
        dataset_quantities=dataset_quantities,
        dataset_type=dataset_type
    )

# === Save models (loop through if you want) ===
for i, model_data in enumerate(all_models):
    save_model(
        folder="models_and_data",
        filename=f"model_additional_{model_data['additional_data']}.pt",
        model=model_data["model"],
        train_losses=model_data["train_losses"],
        train_accs=model_data["train_accs"],
        test_losses=model_data["test_losses"],
        test_accs=model_data["test_accs"],
        additional_data=model_data["additional_data"],
        dataset_type=model_data["dataset_type"]
    )

✅ Dataset saved to models_and_data\dataset.pt
✅ Model saved to models_and_data\model_additional_0.pt
✅ Model saved to models_and_data\model_additional_540.pt
✅ Model saved to models_and_data\model_additional_5940.pt
✅ Model saved to models_and_data\model_additional_59940.pt
